## A simple CNN for text classification

Based on an example from the Keras team, https://github.com/keras-team/keras/blob/master/examples/imdb_cnn.py



This notebook uses a popular neural network API, [Keras](https://keras.io/), to build a simple CNN classifer, and runs it over movie reviews from IMDb - the Internet Movie Database. These reviews are available as a pre-prepared dataset that can be downloaded by the Keras distribution. The dataset is also available from [here](http://ai.stanford.edu/~amaas/data/sentiment/)

The dataset is constructed from very polarised reviews, and has been used in text classification evaluations for several years.

Here's an example positive review:

> I went to an advance screening of this movie thinking I was about to embark on 120 minutes of cheezy lines, mindless plot, and the kind of nauseous acting that made "The Postman" one of the most malignant displays of cinematic blundering of our time. But I was shocked. Shocked to find a film starring Costner that appealed to the soul of the audience. Shocked that Ashton Kutcher could act in such a serious role. Shocked that a film starring both actually engaged and captured my own emotions. Not since 'Robin Hood' have I seen this Costner: full of depth and complex emotion. Kutcher seems to have tweaked the serious acting he played with in "Butterfly Effect". These two actors came into this film with a serious, focused attitude that shone through in what I thought was one of the best films I've seen this year. No, its not an Oscar worthy movie. It's not an epic, or a profound social commentary film. Rather, its a story about a simple topic, illuminated in a way that brings that audience to a higher level of empathy than thought possible. That's what I think good film-making is and I for one am throughly impressed by this work. Bravo!

And here's a negative review example:

> It hurt to watch this movie, it really did... I wanted to like it, even going in. Shot obviously for very little cash, I looked past and told myself to appreciate the inspiration. Unfortunately, although I did appreciate the film on that level, the acting and editing was terrible, and the last 25-30 minutes were severe thumb-twiddling territory. A 95 minute film should not drag. The ratings for this one are good so far, but I fear that the friends and family might have had a say in that one. What was with those transitions? Dear Mr. Editor, did you just purchase your first copy of Adobe Premiere and make it your main goal to use all the goofy transitions that come with that silly program? Anyway... some better actors, a little more passion, and some more appealing editing and this makes a decent movie.


### A note on performance
From the original code comments: This example demonstrates the use of Convolution1D for text classification. It gets to 0.89 test accuracy after 2 epochs. Speed:
* 90s/epoch on Intel i5 2.4Ghz CPU.
* 10s/epoch on Tesla K40 GPU.


### Packages

First, the import - you will need keras, but even though it is not needed as an import, you will also need a neural net backend installed for Keras to use, such as Tensorflow or Theano. Make sure you have one of these available, and make sure it is compatible with the version of Keras you are using. If you use the latest Keras and the latest Tensroflow, you should be ok. I used:

* Tensorflow: 1.3.0
* Keras: 2.2.0

Note that in order for the visualisation to work, you will need to have pydot and graphviz installed, e.g. 

```sudo apt-get install graphviz
pip3 install pydot```

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# For displaying
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt

# For processing example texts into one-hot vectors
import nltk
!pip install numpy==1.16.2
import numpy as np
from nltk.corpus import stopwords
from keras.preprocessing import text



We are going to use numpy's load function to load our IMDb movie review dataset. Unfortunately, a bug in the IMDb python module means it will not load unless we change numpy's load fucntion a little:

In [0]:
#old = np.load
#np.load = lambda *a,**k: old(*a,**k,allow_pickle=True)

### Parameters

Now let's set up some parameters, such as number of features, embedding dimensions, batch size, epocchs etc.

In [0]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

### Data

Let's load the data, and pad it out so all are the same length. In the data each review is labelled with an integer value of either 0 (negative review), or 1 (a positive review).

In [0]:
print('Loading data...\n')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('\nData loaded.')

## Building the model

Next we build the model

In [0]:
print('Build model...\n')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Finished building model.\n')

## Take a look at the model

Keras can print out textual and graphical representations of a model, that tells us:

* The layes in the model, in the order in which they appear in the model
* The output shape - i.e. the size of the matrices passed between layers. In some layers, the final dimension will be the number of units, in CNN laters, it will be the number of filters.
* Parameters - this is the number of weights in each layer

Let's take a look at our model...


In [0]:
print(model.summary())

We can also visualise this

In [0]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Train the model

Now let's train it. Keras will validate against our test data, showing us loss and accuracy as it goes. We will save our metrics so we can display them afterwards.

In [0]:
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


## Visualise the training process

OK, but how did that change over time?
(Thanks to [Jason Brownlee](https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/) for this next bit of code)

In [0]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Using our model

Keras has facility for saving and loading models. For now, we will just use it over an example. We have to pre-process these in to the same one-hot input vectors as used to train our model. We will also remove stopwords - except any that are important to our model. (Credit to [https://github.com/abdel/imdb-sentiment-analysis](Abdelrahman Ahmed) for inspiration).

Try running on different reviews - class 0 are bad reviews, class 1 are good reviews. There are some to get you started below, some made up, some from IMDb, and some recent ones from the Guardian. Edit the line after the example reviews, to change the one being classified.

How does it do? Why? How could you improve it?

In [0]:

# Lists of reviews to try. Good reviews should be class 1, and bad reviews should be class 0
    

bad = [
         'A truly awful movie. I never want to see rubbish like this again. Really bad, worthless, hopeless, bad acting',

         'How do you update Shaft for the modern era? John Singleton tried in 2000 with a serviceable if unspectacular sequel, a rather asexual and anonymous follow-up to the far more stylish and distinctive original. Almost two decades later and somehow we\'re even further from the right answer. Because it turns out that a 2019 version of Shaft probably shouldn\'t turn into an unabashed celebration of regressively misogynistic and homophobic masculinity. In Ride Along director Tim Story\'s wildly misjudged follow-up, we\'re given a Jordan Peterson-level assault on so-called beta millennial males, a strange, angry attack on modernity that feels like the result of a group of bitter men griping about the metrosexualisation of a younger generation. In the latest incarnation, the younger Shaft, played by the undeniably charming Jessie T Usher, is a decaf coffee-drinking, gun-hating, women-respecting data analyst for the FBI, which turns him into a joke and a punchline for both the film and his absentee father. Played by a returning Samuel L Jackson, he\'s disgusted to see what his son has turned into at the hands of his mother (a wasted Regina Hall), who raised him in his absence. Convinced he must be gay, he\'s determined to turn him into his idea of a real man as the pair investigate the death of younger Shaft\'s ex-junkie friend. There\'s an inevitable culture clash between the two but elder Shaft is focused on showing his son that listening less to women and shooting more guns will help to show him the way. There\'s a smart, self-aware film to be made from a rough kernel of this setup. Focusing on the different definitions of masculinity shared by two generations of men is an intriguing entry point, especially given that one is a father who hasn\'t been present for his son\'s youth. There\'s space for suggesting a happy medium between two extremities but Story\'s update has zero interest in nuance or even debate. We\'re shown time and time again that for the younger Shaft, the more he embraces modern, “softer” qualities, the less of a man he then is. Skinny jeans, coconut water, desk work – all treated with unbridled disdain as for his father they all symbolise femininity or, even worse, homosexuality. The film is littered with uneasy jabs not just towards gay men but also the trans community (young Shaft\'s boss complains that his biggest problem is that his daughter “wants to be known as Frank”). “You sure you like pussy?” is repeated by a concerned Jackson with such alarming frequency that I was tempted to ask the critic next to me whether we had somehow been magically transported back to the 70s. But even in the 1971 original, sexual and gender politics were far less troubling. Back then, Shaft even had a gay friend of sorts, a barman who he treated as his equal, but somehow almost 20 years later, every reference to homosexuality is dripping in bile. While 70s Shaft might have been dismissive about the women he was having sex with, he didn\'t feel the need to pause the film to give a sermon about how all women desperately want and need to be treated with unquestioned strength and power, something that the modern incarnation deems necessary. It would be one thing if the film presented him as a dinosaur but the script, from Black-ish creator and Girls Trip co-writer Kenya Barris and Family Guy\'s Alex Barnow, is too busy hero-worshipping him to bother finding fault. The film presents us with two outwardly strong female characters, in Hall and younger Shaft\'s love interest, played by Love, Simon\'s Alexandra Shipp, but any feistiness is soon overruled by their visible arousal at men being men. In one of the film\'s strangest scenes, younger gun-hating Shaft is forced into a shootout as his date watches smiling, turned on by his blood thirst. Similarly Hall\'s date with a beta male (who\'s treated with contempt for having manners and being scared by another shootout) is interrupted by a swaggering Jackson, the man who abandoned her with a baby, making her quiver by showing off the two women he has arrived with and embarrassing her date. Even outside of the script\'s aggressively repetitive bigotry, the shambolic Scooby Doo plot struggles to grab even the slightest amount of attention. There are half-assed attempts to modernise a familiar narrative with references to an Islamic church and men suffering from PTSD but it soon devolves into dull, by-the-numbers, jarringly over-sentimental sitcom. When action arrives it\'s also haphazardly choreographed, especially in a shoddy, confusingly shot finale. Jackson shamelessly showboats throughout but his charisma is buried underneath a shtick that becomes so gratingly obnoxious I almost applauded when Richard Roundtree\'s original Shaft made his inevitable cameo. But rather than saving the day, he\'s given little to do, and the script chooses to defend both admittedly terrible absentee fathers while throwing Hall\'s beleaguered single mother under the bus for feminising her son. It\'s frustrating to see such an underrated comic actor like Hall struggle to find space to shine, although she does provide one of the film\'s rare funny moments as she talks to herself in a public bathroom. Usher similarly, who was so good in the now cancelled TV comedy Survivor\'s Remorse, does his best with a crudely etched character, but any resistance against the film\'s regressive politics is ultimately futile. The mission in Shaft is to break down a modern definition of masculinity, to toughen up “delicate” qualities, such as hating guns and listening to women, and return to the good old days instead. While Jackson\'s ribald relic might succeed in forcing his son back to the past, this embarrassingly tone-deaf film fails to take us with him.',

         'It hurt to watch this movie, it really did... I wanted to like it, even going in. Shot obviously for very little cash, I looked past and told myself to appreciate the inspiration. Unfortunately, although I did appreciate the film on that level, the acting and editing was terrible, and the last 25-30 minutes were severe thumb-twiddling territory. A 95 minute film should not drag. The ratings for this one are good so far, but I fear that the friends and family might have had a say in that one. What was with those transitions? Dear Mr. Editor, did you just purchase your first copy of Adobe Premiere and make it your main goal to use all the goofy transitions that come with that silly program? Anyway... some better actors, a little more passion, and some more appealing editing and this makes a decent movie.'
      ]
    
good = [
         'I loved this movie. The best I have seen this year. Great acting, brilliant plot, wonderful dialogue.',

         'I went to an advance screening of this movie thinking I was about to embark on 120 minutes of cheezy lines, mindless plot, and the kind of nauseous acting that made "The Postman" one of the most malignant displays of cinematic blundering of our time. But I was shocked. Shocked to find a film starring Costner that appealed to the soul of the audience. Shocked that Ashton Kutcher could act in such a serious role. Shocked that a film starring both actually engaged and captured my own emotions. Not since "Robin Hood" have I seen this Costner: full of depth and complex emotion. Kutcher seems to have tweaked the serious acting he played with in "Butterfly Effect". These two actors came into this film with a serious, focused attitude that shone through in what I thought was one of the best films I\'ve seen this year. No, its not an Oscar worthy movie. It\'s not an epic, or a profound social commentary film. Rather, its a story about a simple topic, illuminated in a way that brings that audience to a higher level of empathy than thought possible. That\'s what I think good film-making is and I for one am throughly impressed by this work. Bravo!',

         'A documentary about the pain of mothers losing a connection with their children might not sound like one of the most uplifting films of the year, but Kristof Bilsen\'s film is a radical achievement: a love letter to loss, sacrifice and yearning. It questions how we care for elderly loved ones, makes provocative contrasts between east and west in the economics of medicine, and, with a central character who\'s pure charisma, this is intimate observational documentary-making of a high standard. Pomm is a carer in Thailand for westerners with Alzheimer\'s. She gives her patient one-to-one care, which comprises singing, joking, hugging and confiding, as well as the basics of cleaning and welfare. This is more personal attention than would be possible in her patients\' home countries, and though it doesn\'t come cheap, families feel it\'s worth the expense of sending someone halfway across the world for it. When we first meet her, Pomm\'s patient is Elisabeth, who can communicate only in squeaks and other noises, but seems calm and content. Pomm has her own problems, primarily lack of contact with her children who live many miles away. Tender late-night confessions to Elisabeth make clear the extent to which the support is mutual. This is a brilliantly novel way to establish characters\' motivations and doesn\'t feel crass. It\'s natural and born of their trust in Bilsen, whose presence in the room appears to be minimal. Pomm and Elisabeth are each other\'s family, bound by their need for one another. A segue to Pomm\'s relationship with her children feels uncomfortable – she\'s out of their sight and out of their minds, and it hurts. Later, Pomm has another patient, Maya, brought from Switzerland by a family who love her but have decided the greatest love is to let her go and put her in the hands of a stranger. With Maya\'s arrival in Thailand, the film shifts up a gear, moving from tenderness and sensitivity to something much harder to watch, and Bilsen deserves praise for doing it. Earlier scenes with Maya and her family in the mountains of Switzerland are puzzling and even alienating, as we are plunged into their world without any context. We\'re intrigued but feel like intruders on their private foreboding as they know their time together is running out. Bilsen\'s plotting from start to finish is immaculate, never explaining too much but always hinting just enough about the melancholy destinies of the characters. Mother explores pain and dignity, and the way that some people\'s pain is considered more important than others. Pomm has lower status, only begrudgingly granted time off by her boss to see her family. No one asks her about her wellbeing, no one dotes over her. Yet she is the hero of this piece, giving everything for strangers, night and day, in an exhausting service so her own family can be cared for. This is an original piece of work, addressing the selflessness of mothers and the impact of dementia on families. These subjects have been the focus of documentaries before but rarely in this combination and with such an unflinching resolve to keep filming in uncomfortable moments.exampleText'
   
       ]
    

# Change the line below to classify one of the examples above.
example = good[1]
print(example)

# Download NLTK stopwords
nltk.download('stopwords')

# Prepare the stopwords
stopwords_nltk = set(stopwords.words('english'))
relevant_words = set(['not', 'nor', 'no', 'wasn', 'ain', 'aren', 'very', 'only', 'but', 'don', 'isn', 'weren'])
stopwords_filtered = list(stopwords_nltk.difference(relevant_words))

# Remove the stop words from input text
example = ' '.join([word for word in example.split() if word not in stopwords_filtered])

# One-hot the input text
example = text.one_hot(example, max_features)
example = np.array(example)

# Pad the sequences
example = sequence.pad_sequences([example], maxlen=maxlen)

# Make the prediction
pred_prob = model.predict(example)[0][0]
pred_class = model.predict_classes(example)[0][0]

print("Probability: ", pred_prob)
print("Class: ", pred_class)

